<a href="https://colab.research.google.com/github/WesleySantosMaxtelll/doutorado/blob/main/MoE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install torch
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 122.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import collections
import matplotlib.pyplot as plt
import sys
import ast
import pandas as pd
import torch
import os
import torch.nn.functional as F
import numpy as np
import torch.nn as nn
from torch.optim import AdamW, SGD
from transformers import BertTokenizerFast, BertModel, DistilBertTokenizer, DistilBertModel
from torch.nn.utils.rnn import pack_padded_sequence
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, Dataset
from sklearn.utils import class_weight
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import _pickle as pickle
import time
from datetime import datetime, timedelta
import gc
from torch.nn import CrossEntropyLoss
from tqdm import tqdm
import random

In [ ]:

def load_data(portion, task, sel):
    df_c = pd.read_csv(f'/content/drive/MyDrive/doutorado/dados_v6/full/{task}/{portion}_{task}_c_SetembroBR_v6.csv', sep=';')
    df_d = pd.read_csv(f'/content/drive/MyDrive/doutorado/dados_v6/full/{task}/{portion}_{task}_SetembroBR_v6.csv', sep=';')
    df_c_net = pd.read_csv(f'/content/drive/MyDrive/doutorado/dados_v6/full/{task}/{task}_ctrl-users-network_final_v6.csv', sep=';')
    df_d_net = pd.read_csv(f'/content/drive/MyDrive/doutorado/dados_v6/full/{task}/{task}_diag-users-network_final_v6.csv', sep=';')
    if sel not in ['all', 'all_br', 'balanced', 'medical_terms', 'pref_medical_terms', 'r80', 'dev80', 'r100']:  
        df_d = df_d[df_d['User_ID'].isin(df_d_net[df_d_net['Event_Type'] == sel]['User_ID'])]
        df_c_net = df_c_net[df_c_net['Counterpart'].isin(df_d['User_ID'])]
        df_c = df_c[df_c['User_ID'].isin(df_c_net['User_ID'])]
    if sel == 'balanced' and portion == 'train':
        # return df_c, df_d, df_c_net
        dt = {'D':'depression', 'A': 'anxiety'}
        user_id_c_all = []
        for _, user_id in df_d.iterrows():
            user_id_c = df_c_net[df_c_net['Counterpart'] == user_id['User_ID']].sample(n=1).iloc[0]['User_ID']
            user_id_c_all.append(user_id_c)
        df_c = df_c[df_c['User_ID'].isin(set(user_id_c_all))] 
    return pd.concat((df_c, df_d))


In [ ]:
import re

# filter medical terms
def makeTags(text):
    text = str(text).lower()
    if re.search(r"diagn[oó]stic", text, flags=re.DOTALL):
        return 'diagnostico'
    if re.search(r"tratamento", text, flags=re.DOTALL):
        return 'tratamento'
    if re.search(r"depress[aã]o", text, flags=re.DOTALL):
        return 'depressao'
    if re.search(r"ansiedade", text, flags=re.DOTALL):
            return 'ansiedade'
    if re.search(r"depressivo|tarja\sp|ansiol[íi]tico", text, flags=re.DOTALL):
        return 'antidepress'
    if re.search(r"m[eé]dic[oa]|psic[oó]l[oó]g[oa]|psiquiatra|neurologista|terapeuta", text, flags=re.DOTALL):
        return 'medico'
    return ''

def HasMeds(text):
    text = str(text).lower()
    terms = r'(hospital|cl[ií]nic|depr[eê]|m[eé]dic[oa]|psic[oó]log[oa]|psiquiatra|neurologista|\bterapeuta|\bterapia|rem[ée]dio|medicamento|medica[çc][aã]o|depress[aã]o|ansiedade|anti-depressivo|depressivo|tarja\sp|ansiol[íi]tico|escitalopran|amitriptilina|sertralina|citalopram|fluoxetina)'
    if re.search(terms, text, flags=re.DOTALL):
        return 1
    else:
        return 0

def filter_medical_terms(texts):
    sel_texts_ = []
    for text_list in tqdm(texts):
        sel_texts = [tt for tt in text_list if makeTags(tt) == '']
        sel_texts_.append(sel_texts)
    return sel_texts_


def sel_medical_terms(texts):
    size = 100
    sel_texts_ = []
    for text_list in tqdm(texts):
        sel_texts = [tt for tt in text_list if makeTags(tt) == '']
        med_sel_texts = [tt for tt in text_list if makeTags(tt) != '']
        if size <= len(med_sel_texts):
            sel_texts_.append(med_sel_texts[-size:])
        else:
            l = size - len(med_sel_texts)
            # print(l)
            if len(sel_texts) > l:
                add_texts = random.sample(sel_texts, l)
                med_sel_texts = med_sel_texts + add_texts
                random.shuffle(med_sel_texts)
                sel_texts_.append(med_sel_texts)
            else:
                sel_texts_.append(med_sel_texts + sel_texts)

    return sel_texts_
  
def random_cont_selection(texts, n=80):
    sel_texts_ = []
    for text_list in tqdm(texts):
        if len(text_list) > n + 1:
            fp = random.randint(n, len(text_list))
            sp = fp - n
        else:
            sp = 0
            fp = len(text_list)
        sel_texts_.append(text_list[sp:fp])

    return sel_texts_

def recent_selection(texts, n=80):
    sel_texts_ = []
    for text_list in tqdm(texts):
        sel_texts_.append(text_list[-n:])

    return sel_texts_


In [ ]:
def load_convert(task, sel):
    df_test = load_data('test', task, sel)
    df_train = load_data('train', task, sel)
    df_test['Text'] = df_test['Text'].apply(lambda x: x.split('#'))
    df_test['label'] = df_test['Diagnosed_YN'].apply(lambda x: 1 if x == 'yes' else 0)
    df_train['Text'] = df_train['Text'].apply(lambda x: x.split('#'))
    df_train['label'] = df_train['Diagnosed_YN'].apply(lambda x: 1 if x == 'yes' else 0)
    train_initial_size = df_train['Text'].apply(lambda x: len(x)).sum()
    test_initial_size = df_test['Text'].apply(lambda x: len(x)).sum()
    random.seed(42)
    if sel == 'medical_terms': 
        print('Start filter')
        df_train['Text'] = filter_medical_terms(df_train['Text'])
        df_test['Text'] = filter_medical_terms(df_test['Text'])
        train_final_size = df_train['Text'].apply(lambda x: len(x)).sum()
        test_final_size = df_test['Text'].apply(lambda x: len(x)).sum()
        print(f'On task {task}.\n\tThe number of samples in the train dataset is ')
    elif sel == 'pref_medical_terms': 
        print('sel', sel)
        df_train['Text'] = sel_medical_terms(df_train['Text'])
        df_test['Text'] = sel_medical_terms(df_test['Text'])
        train_final_size = df_train['Text'].apply(lambda x: len(x)).sum()
        test_final_size = df_test['Text'].apply(lambda x: len(x)).sum()
        print(f'On task {task}.\n\tThe number of samples in the train dataset is ', len(df_train))
    elif sel == 'r80':
        print('sel', sel)
        df_train['Text'] = random_cont_selection(df_train['Text'])
        # df_test['Text'] = random_cont_selection(df_test['Text'])
    elif sel == 'r100':
        print('sel', sel)
        df_train['Text'] = random_cont_selection(df_train['Text'], 100)
        # df_test['Text'] = random_cont_selection(df_test['Text'])
    elif sel == 'dev80':
        print('sel', sel)
        df_train['Text'] = recent_selection(df_train['Text'])
        # df_test['Text'] = random_cont_selection(df_test['Text'])
    return df_train, df_test

In [ ]:
import random

class CustomTextDataset(Dataset):
    def __init__(self, tokens_mask, labels):
        self.tokens_mask = tokens_mask
        self.labels = labels
        self.size = 10
        random.seed(42)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        label = self.labels[idx]
        data = self.tokens_mask[idx]
        start = random.randint(0, data['tokens'].shape[0] - self.size)
        data_short = {v: data[v][start: start + self.size] for v in data}
        return [data_short, label]

class CustomTextDatasetText(Dataset):
    def __init__(self, tokens_mask, texts, labels):
        self.tokens_mask = tokens_mask
        self.labels = labels
        self.texts = texts
        self.size = 10
        random.seed(42)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        label = self.labels[idx]
        data = self.tokens_mask[idx]
        start = random.randint(0, data['tokens'].shape[0] - self.size)
        data_short = {v: data[v][start: start + self.size] for v in data}
        print(type(self.texts[idx]), self.texts[idx])
        # print(type(self.texts[idx]), self.texts[idx])
        texts = self.texts[idx][start: start + self.size]
        # print(texts)
        return [data_short, texts, label]


In [ ]:
torch.cuda.empty_cache()
device = torch.device('cuda')
print(device)
source_folder = 'Data'
destination_folder = 'Model'


# Define the function for creating a weight dictionary.
def create_weight_dict(labels):
    unique_labels = np.unique(labels)
    class_weights = class_weight.compute_class_weight('balanced', classes=unique_labels, y=labels)
    return class_weights


def get_loss(outputs, b_labels, vect_weights):
    loss_fct = CrossEntropyLoss(weight=torch.tensor(vect_weights, dtype=torch.float32))
    loss_fct.to(device)
    loss = loss_fct(outputs[1].view(-1, 2), b_labels.view(-1))

    return loss

cuda


In [ ]:
class ExpertGate(nn.Module):
    def __init__(self, device, s, output_size=2):
        super(ExpertGate, self).__init__()
        self.device = device
        self.hidden_size = s
        self.rnn = nn.LSTM(input_size=self.hidden_size, hidden_size=100,
                           num_layers=4, bidirectional=True)
        self.sample_size = 10
        self.fc_out3 = nn.Linear(6000, 1000)
        self.fc_out2 = nn.Linear(1000, 100)
        self.fc_out1 = nn.Linear(100 * self.sample_size, output_size)

    def forward(self, bert_vectors):
        outputs = []
        # iterate in the batch through all sequences
        for encoded in bert_vectors:
            # encoded = self.bert(input_ids=token, attention_mask=mask)['last_hidden_state']
            out, hiden = self.rnn(encoded)
            out = self.fc_out3(out.view(self.sample_size, -1))
            out = F.dropout(out, 0.5)
            out = self.fc_out2(out)
            out = F.dropout(out, 0.5)
            out = self.fc_out1(out.view(-1))
            out = torch.softmax(out, -1)
            outputs.append(out.squeeze())
        outputs = torch.stack(outputs)
        return outputs

class MoE(nn.Module):
    def __init__(self, device, exps=3):
        super(MoE, self).__init__()
        pre_trained = 'neuralmind/bert-base-portuguese-cased'
        self.bert = BertModel.from_pretrained(pre_trained)
        self.exps = exps
        self.device = device
        self.nets = nn.ModuleList([
          ExpertGate(device, self.bert.config.hidden_size) for _ in range(self.exps)
        ])
        self.gating = ExpertGate(device, self.bert.config.hidden_size, output_size=exps)
    
    def forward(self, atts, masks):
        bert_out = []
        for token, mask in zip(atts, masks):
            # print(1)
            bert_vectors = self.bert(input_ids=token, attention_mask=mask)['last_hidden_state']
            bert_out.append(bert_vectors)
        # print(2)
        weights_sof = self.gating(bert_out)
        # print(3)
        preds = []
        for n in self.nets:
            # print(4)
            preds.append(n(bert_out))
        # print(5)
        return (torch.stack(preds) * weights_sof.T[..., None]).sum(0)

class MoEBR(nn.Module):
    def __init__(self, device, exps=3):
        super(MoEBR, self).__init__()
        pre_trained = 'pablocosta/bertweet-br-base-uncased'
        self.bert = BertModel.from_pretrained(pre_trained)
        self.exps = exps
        self.device = device
        self.nets = nn.ModuleList([
          ExpertGate(device, self.bert.config.hidden_size) for _ in range(self.exps)
        ])
        self.gating = ExpertGate(device, self.bert.config.hidden_size, output_size=exps)
    
    def forward(self, atts, masks):
        bert_out = []
        for token, mask in zip(atts, masks):
            # print(1)
            bert_vectors = self.bert(input_ids=token, attention_mask=mask)['last_hidden_state']
            bert_out.append(bert_vectors)
        # print(2)
        weights_sof = self.gating(bert_out)
        # print(3)
        preds = []
        for n in self.nets:
            # print(4)
            preds.append(n(bert_out))
        # print(5)
        return (torch.stack(preds) * weights_sof.T[..., None]).sum(0)
    

In [ ]:
from tqdm import tqdm

def get_tokens_mask_dates_split(X=None, y=None, tokenizer=None, type_data=None, task=None, sel=None, ml=30):
    path = f'/content/drive/MyDrive/doutorado/diag_trat_saving/{type_data}_{sel}_{task}_processed.pkl'

    if os.path.isfile(path):
        print('load', path)
        # 0/0
        X_processed, y = pickle.load(open(path, 'rb'))
    else:
        print('Prepare')
        X_processed = []
        for user_posts in tqdm(X):
            # print(tokenizer)
            user_posts = user_posts[-1500:]
            tokenized_texts = tokenizer.batch_encode_plus(user_posts, max_length=ml, padding='max_length', truncation=True)
            tokens_tensor = torch.tensor(tokenized_texts['input_ids'])
            segments_tensors = torch.tensor(tokenized_texts['attention_mask'])
            X_processed.append(
                {'tokens': tokens_tensor,
                 'mask': segments_tensors,
                 }
            )
        pickle.dump([X_processed, y], open(path, 'wb'))
    return X_processed, y
  

In [ ]:
def save_checkpoint(save_path, model, valid_loss):
    if save_path == None:
        return

    state_dict = {'model_state_dict': model.state_dict(),
                  'valid_loss': valid_loss}

    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')

In [ ]:
def get_dataloader(X_, y_, shuffle):
    batch_size = 16
    TD_ = CustomTextDataset(X_, y_)
    return DataLoader(TD_, batch_size=batch_size, shuffle=shuffle, collate_fn=(lambda x: x))

def get_dataloader_text(X_, X_raw, y_, shuffle):
    batch_size = 16
    TD_ = CustomTextDatasetText(X_, X_raw, y_)
    return DataLoader(TD_, batch_size=batch_size, shuffle=shuffle, collate_fn=(lambda x: x))


In [ ]:
def load_checkpoint(load_path, model):
    if load_path == None:
        return

    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')

    model.load_state_dict(state_dict['model_state_dict'])
    return state_dict['valid_loss']

In [ ]:
from sklearn.model_selection import train_test_split


def run_train(task, sel, model_name):
    print(task, sel, model_name)
    try:
        print('Load')
        # 1/0
        X_train, y_train = get_tokens_mask_dates_split(None, None, None, 'train', task, sel)
        X_test, y_test = get_tokens_mask_dates_split(None, None, None, 'test', task, sel)    
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    except:
        print('Create')
        df_train, df_test = load_convert(task, sel)
        print(len(df_train), len(df_test))   
        # return df_train, df_test
        # tokenizer = BertTokenizerFast.from_pretrained('neuralmind/bert-base-portuguese-cased')
        tokenizer = BertTokenizerFast.from_pretrained('pablocosta/bertweet-br-base-uncased')
        X_train, y_train = get_tokens_mask_dates_split(df_train['Text'].tolist(), df_train['label'].tolist(), tokenizer, 'train', task, sel)
        X_test, y_test = get_tokens_mask_dates_split(df_test['Text'].tolist(), df_test['label'].tolist(), tokenizer, 'test', task, sel)    
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    dataloader_train = get_dataloader(X_train, y_train, True)
    dataloader_val = get_dataloader(X_val, y_val, True)
    dataloader_test = get_dataloader(X_test, y_test, False)

    device = 'cuda'
    if model_name == 'moe':
        model = MoE(device)
    elif model_name == 'moe_br':
        model = MoEBR(device)
    else:
        raise Exception('Model not defined')

    model.to(device)

    print('*' * 20)
    
    lr = 1e-5
    optimizer = AdamW(model.parameters(), lr=lr)

    train_loss_set = []
    epochs = 50
    file_path = f'/content/drive/MyDrive/doutorado/diag_trat_saving/{sel}/'
    vect_weights = create_weight_dict(y_train)
    print(vect_weights)
    loss_fct = CrossEntropyLoss(weight=torch.tensor(vect_weights, dtype=torch.float32), reduction='mean')
    loss_fct.to(device)
    for epoch in range(epochs):
        print(f'Epoch {epoch}')
        model.train()
        pred, true_label = [], []
        print(len(dataloader_train))
        for i, batch in enumerate(dataloader_train):
            if i % 100 == 0:
                print(datetime.fromtimestamp(time.time()), i, train_loss_set[-1] if len(train_loss_set) else 0)
            atts = torch.stack([aa[0]['tokens'] for aa in batch]).to(device)
            masks = torch.stack([aa[0]['mask'] for aa in batch]).to(device)
            outputs = model(atts, masks)
            loss = loss_fct(outputs, torch.tensor([b[1] for b in batch]).to(device))
            # output = loss(torch.argmax(outputs[1], dim=1).add_(-1), b_labels.add_(-1))
            train_loss_set.append(loss.item())
            # Backward pass
            loss.backward()
            # Update parameters and take a step using the computed gradient
            optimizer.step()
            # scheduler.step()
            optimizer.zero_grad()
            
            pred = pred + outputs.cpu().argmax(1).tolist()
            true_label = true_label + [b[1] for b in batch]
            gc.collect()
            torch.cuda.empty_cache()
        print('Results')
        print(classification_report(true_label, pred))
        # continue
        model.eval()
        with torch.no_grad():
            # validation loop
            pred, real = [], []
            for j, batch in enumerate(dataloader_val):
                atts = torch.stack([aa[0]['tokens'] for aa in batch]).to(device)
                masks = torch.stack([aa[0]['mask'] for aa in batch]).to(device)
                if model_name == 'ens':
                    outputs, model_outputs = model(atts, masks)
                else:
                    outputs = model(atts, masks)

                pred = pred + outputs.cpu().argmax(1).tolist()
                real = real + [b[1] for b in batch]
            print(classification_report(real, pred))
            save_checkpoint(file_path + '/' + f'model-{model_name}-{task}-sel-{sel}-epoch{epoch}.pt', model, 0)
        model.train()



In [ ]:
# Run this cell to train the selected models

import itertools

tasks = ['A', 'D'] # 'A' to anxiety and 'D' to depression
selection = ['all_br'] # The selection of data
models = ['moe_br'] # model name

for task, sel, model_name in itertools.product(tasks, selection, models):
    run_train(task, sel, model_name)

A all_br bert_twitter
Load
load /content/drive/MyDrive/doutorado/diag_trat_saving/train_all_br_A_processed.pkl
load /content/drive/MyDrive/doutorado/diag_trat_saving/test_all_br_A_processed.pkl


Some weights of the model checkpoint at pablocosta/bertweet-br-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


********************
[0.57315843 3.91724138]
Epoch 0
710
2022-08-26 15:54:00.119235 0 0
2022-08-26 15:55:59.523579 100 0.6958425045013428
2022-08-26 15:57:58.077085 200 0.7556463479995728
2022-08-26 15:59:56.677386 300 0.643033504486084
2022-08-26 16:01:55.224315 400 0.6591290831565857
2022-08-26 16:03:53.691870 500 0.6274153590202332
2022-08-26 16:05:52.052942 600 0.7589823603630066
2022-08-26 16:07:50.383814 700 0.6568059325218201
Results
              precision    recall  f1-score   support

           0       0.88      0.83      0.86      9910
           1       0.17      0.23      0.19      1450

    accuracy                           0.76     11360
   macro avg       0.52      0.53      0.53     11360
weighted avg       0.79      0.76      0.77     11360

              precision    recall  f1-score   support

           0       0.91      0.85      0.88      2515
           1       0.24      0.36      0.29       325

    accuracy                           0.79      2840
   macro a

KeyboardInterrupt: ignored

In [ ]:
import re 
import gc
import itertools

# Make the prediction with a specific model on 50 random sequences for each timeline
def make_prediction(model, dataloader_, model_name):
    model.eval()
    with torch.no_grad():
        # validation loop
        all_pred = []
        for _ in range(50):
            pred, real = [], []
            for j, batch in enumerate(dataloader_):
                atts = torch.stack([aa[0]['tokens'] for aa in batch]).to(device)
                masks = torch.stack([aa[0]['mask'] for aa in batch]).to(device)
                
                if model_name == 'ens':
                    outputs, model_outputs = model(atts, masks)
                else:
                    outputs = model(atts, masks)
                # outputs = model(atts, masks)

                pred = pred + outputs.cpu().argmax(1).tolist()
                real = real + [b[1] for b in batch]
            print(classification_report(real, pred))
            all_pred.append(pred)

        p = (np.array(all_pred).mean(0) > 0.5).astype(int)
        print("*" * 20, '\nALL')
        print(classification_report(real, p))
        print("*" * 20)


# Receives the task, data selection and model name
def evaluate_results(task, sel, model_name):
    gc.collect()
    device = 'cuda'
    if model_name == 'moe':
        model = MoE(device)
    elif model_name == 'moe_br':
        model = MoEBR(device)

    else:
        raise Exception('Model not defined')

    model.to(device)
    file_path = f'/content/drive/MyDrive/doutorado/diag_trat_saving/{sel}/'
    files = os.listdir(file_path)
    regex = re.compile(f'model-{model_name}-{task}-sel-{sel}-epoch')
    files = [f for f in files if regex.search(f)]
    df_train, df_test = load_convert(task, sel)    
    # tokenizer = BertTokenizerFast.from_pretrained('neuralmind/bert-base-portuguese-cased')
    tokenizer = BertTokenizerFast.from_pretrained('pablocosta/bertweet-br-base-uncased')
    X_train, y_train = get_tokens_mask_dates_split(df_train['Text'].tolist(), df_train['label'].tolist(), tokenizer, 'train', task, sel)
    # X_test, y_test = get_tokens_mask_dates_split(df_test['Text'].tolist(), df_test['label'].tolist(), tokenizer, 'test', task, sel)    
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    del X_train
    del y_train
    del df_test
    gc.collect()
    dataloader_val = get_dataloader(X_val, y_val, False)
    # dataloader_test = get_dataloader(X_test, y_test, False)
    
    # define the epoch you want to run
    for f in [15, 18, 19, 29, 31, 33, 36]:
        path = f'{file_path}model-{model_name}-{task}-sel-all_br-epoch{f}.pt'
        print('*' * 20, f'\n{path}')
        try:
            load_checkpoint(f'{path}', model)
        except:
            continue
        model.eval()
        make_prediction(model, dataloader_val, model_name)
    del df_train
    del X_val
    del y_val
    del dataloader_val
    gc.collect()



# tasks = ['A']
# selection = ['all_br']
# models = ['bert_twitter']

# for task, sel, model_name in itertools.product(tasks, selection, models):
#     # print(task, sel, model_name)
#     evaluate_results(task, sel, model_name)